## Home and work location mapping 
The purpose of this notebook is to map the  home and work locations for the users that passed qc. Specifically we want to map: 

1) All the home locations of users in Bogota 
2) All the work locations of the users in Bogota
3) All the home and work locations of the users that live near the stations

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from plotting import * 
from preprocess import *

import pandas as pd 

# Access environment variables and define other necessary variables
data_dir = os.getenv('WORKING_DIR')
meta_dir = f'{data_dir}metadata/'
pq_dir = f'{data_dir}data/parquet/in_study_area/pass_qc/'
out_dir_hw = f'{data_dir}data/home_work/'


### Load the pings data and map each users' home location

Load 1. the shapefile with the ZAT and stratum information and 2. the user stats to get the list of users. 
Note that there are 701961 users that passed qc. Takes quite some time to run.

In [ ]:
shp_name_zat = 'zat_stratum'
shapefile_zat = f'{meta_dir}income/{shp_name_zat}.shp'
regions_gdf_zat = gpd.read_file(shapefile_zat)
regions_gdf_zat.plot()

output_filepath = f'{data_dir}/data/user_stats/user_stats_2019_months1-8_60min_pings_10min_days_shp_filtered.csv'
user_stats_filtered = pd.read_csv(output_filepath)
uids_pass_qc= list(user_stats_filtered['uid'])
user_stats_filtered.head()

In [ ]:
out_dir = f'{pq_dir}home_locs/'
cols = ['uid', 'datetime', 'lat', 'lng']
gdf_cols = ['Area', 'MUNCod', 'NOMMun', 'ZAT', 'UTAM', 'stratum']

compute_home_lat_lngs_for_users(uids_pass_qc=uids_pass_qc, regions_gdf=regions_gdf_zat, pq_dir=pq_dir, 
                                out_dir=out_dir, num_users=50000, cols = ['uid', 'datetime', 'lat', 'lng'], 
                                gdf_cols=['Area', 'MUNCod', 'NOMMun', 'ZAT', 'UTAM', 'stratum'])

### Merge the home locs from all users to find those that live in the zats of interest 

#### Combine the data for all the users
Read in the home locs files with the home locations of all the users (lat, lng, ZAT) that we just wrote to get the uids of those users that live in those areas

In [ ]:
home_locs_files = glob.glob(out_dir + '*.parquet')
all_home_locs_filepath = f'{out_dir}all_home_locs_2200_0600_w_zats_for_users_pass_qc.csv'
hl_df = ds.dataset(home_locs_files, format="parquet").to_table().to_pandas()
hl_df = hl_df.rename(columns={'lat': 'lat_home', 'lng': 'lng_home', 'ZAT': 'ZAT_home', 'UTAM': 'UTAM_home'})
hl_df.to_csv(all_home_locs_filepath)
hl_df.head()

#### Read in zats file 
This file specifies the treatment and control zats. Make sure that the ZATs are the same data type in the `hl_df` and the `zats_tc` dataframes for proper filtering.

In [ ]:
import numpy as np 

zats_tc_fp = f'{meta_dir}ZAT_treat_control.csv'
print(hl_df.ZAT_home.dtypes)
zats_tc = pd.read_csv(zats_tc_fp).astype('float64') 

ztreat = [i for i in list(zats_tc['ZATs Treatment group']) if str(i) != "nan"] 
zcontrol = [i for i in list(zats_tc['ZAT Control group']) if str(i) != "nan"] 
zsel = ztreat + zcontrol
zats_tc.head()

#### Filter the users by the selected zat ids to get the uids of those users that live in those areas
Also write out a file with data on the homes and stratum, etc of these users.

In [ ]:
hl_df_zsel = hl_df[hl_df['ZAT_home'].isin(zsel)].copy().reset_index()
hl_df_zsel = hl_df_zsel.drop(columns=['index'])

zsel_home_locs_filepath = f'{out_dir}selected_txt_control_home_locs_2200_0600_w_zats_for_users_pass_qc.csv'
hl_df_zsel.to_csv(zsel_home_locs_filepath)
hl_df_zsel.head()

In [ ]:
users_living_in_zsels = list(hl_df_zsel['uid'])

There are 39,655 users that potentially live in one of the 96 ZATs according to where they have the most pings between 10pm and 6am (which is how skmob calculates home locations)

### Filter the ping data to only include pings from those users that live in the specified ZATs
Takes ~2.5 min to filter the data and write all the pq files and 5 min to write the csv 

In [ ]:
pq_dirs_pass_qc = glob.glob(pq_dir + '*.parquet')
pq_dirs_pass_qc_names = [i.split(f'{pq_dir}')[1].split('.parquet')[0] for i in pq_dirs_pass_qc]

for i in tqdm(range(0,len(pq_dirs_pass_qc)), desc=f'Writing data for users that pass qc'):
    print(f'Filtering data for {pq_dirs_pass_qc_names[i]}...')
    dataset = ds.dataset(pq_dirs_pass_qc[i], format="parquet")
    table = dataset.to_table(filter=ds.field('uid').isin(users_living_in_zsels))
    data_for_users_in_zats = f'{pq_dir}/in_zats/{pq_dirs_pass_qc_names[i]}_selected_zats.parquet'
    pq.write_table(table, data_for_users_in_zats)

#### Make a csv file for the pings for all the users in the ZATs

In [ ]:
in_zats_dir = f'{pq_dir}/in_zats/'
in_zats_dir_pq = glob.glob(in_zats_dir + '*.parquet')
in_zats_pings_df = ds.dataset(in_zats_dir_pq, format="parquet").to_table().to_pandas().reset_index()
in_zats_pings_df = in_zats_pings_df.drop(columns=['index'])
in_zats_pings_df.head()


In [ ]:
in_zats_pings_df_filepath = f'{in_zats_dir}pings_home_locs_2200_0600_w_zats_for_users_pass_qc_living_in_selzats.csv'
in_zats_pings_df.to_csv(in_zats_pings_df_filepath)